In [1]:
from shaolin_graphs import *
from shaolin.core.dashboard import ToggleMenu

In [2]:
class GraphPlotBokeh(ToggleMenu):
    
    def __init__(self, gc, name='scatter_plot', **kwargs):
        self._width = 1200
        self._height = 800
        self._plot_title = "Graph plot"
        node_mapper_dict = {'size':{'max':100,
                            'min':1,
                            'step':0.5,
                            'high':75,
                            'low':25,
                            'default':12,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_width':{'max':50,
                            'min':0,
                            'step':0.5,
                            'high':5,
                            'low':1,
                            'default':2,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'fill_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_color':{'default_color':'black','map_data':False,'step':0.05,'min':0.0,'low':0.0},
                       'fill_color':{'default_color':'#11D4CA','map_data':False,'step':0.05,'min':0.0,'low':0.0}
                      }
        edge_mapper_dict = {'size':{'max':100,
                            'min':5,
                            'step':0.5,
                            'high':20,
                            'low':7,
                            'default':7,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_width':{'max':50,
                            'min':0,
                            'step':0.5,
                            'high':5,
                            'low':1,
                            'default':2,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'fill_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_alpha':{'max':1.0,
                            'min':0.,
                            'step':0.05,
                            'high':0.95,
                            'low':0.3,
                            'default':1.,
                            'map_data':False,
                            'fixed_active':False,
                           },
                       'line_color':{'default_color':'black','map_data':False,'step':0.05,'min':0.0,'low':0.0},
                       'fill_color':{'default_color':'#EDB021','map_data':False,'step':0.05,'min':0.0,'low':0.0}
                      }
        gc.name = 'gc'
        output_notebook(hide_banner=True)
        edge_mapper_data = gc.edge
        node_mapper = PlotMapper(gc.node, mapper_dict=node_mapper_dict, name='node_mapper', mode='interactive')
        edge_mapper = PlotMapper(edge_mapper_data, mapper_dict=edge_mapper_dict, name='edge_mapper', mode='interactive')
        node_tooltip_data = gc.node
        node_tooltip = BokehDataFrameTooltip(node_tooltip_data, name='node_tooltip')
        edge_tooltip = BokehDataFrameTooltip(edge_mapper_data, name='edge_tooltip')
        
        ToggleMenu.__init__(self,
                            children=[Dashboard(["r$n=buton_dash",["btn$d=Update&n=update_btn",'tog$d=Plot text']],name='plot_ctrl'),gc,node_mapper,
                                            edge_mapper,
                                            node_tooltip,
                                            edge_tooltip],
                            name=name,**kwargs)
        #self._init_layout()
        #self.observe(self.update)
        self.gc.calculate.observe(self.update)
        self.node_mapper.marker.marker_type.observe(self._on_marker_change)
        self.edge_mapper.marker.marker_type.observe(self._on_marker_change)
        self.plot_ctrl.plot_text.observe(self._on_marker_change)
        self.plot_ctrl.update_btn.observe(self._on_marker_change)
        self._handler = None
    
    def prepare_edge_mapper_data(self,gc):
        et = gc.edge.copy()
        cols = et.items.values
        fi = pd.DataFrame(et[cols[0]].unstack().copy())
        fi.columns = [cols[0]]

        for it in cols[1:]:
            new = pd.DataFrame(et[it].unstack().copy())
            new.columns = [it]
            fi = fi.combine_first(new)
        fi.index.levels[0].name = 'major'
        fi.index.levels[1].name = 'minor'
        dft = fi.reset_index()
        
        ti = list(zip(dft.minor.values,dft.major.values))
        dft.index = ti
        dft = dft.drop(['major','minor'],axis=1)
        return dft.ix[gc.G.edges()].copy()
        
    def _init_layout(self):
        params = ['size', 'line_width', 'fill_alpha', 'line_alpha', 'line_color', 'fill_color']
        for target in ['node','edge']:
            mapper = getattr(self,target+'_mapper')
            getattr(mapper).data_slicer.index_slicer.visible = False
            
    def update_source_df(self):
        """Datasources for plots managing"""
        def index_to_columns(df):
            old_index = df.index
            new_index = pd.MultiIndex.from_tuples(df.index)
            df.index = new_index
            df = df.reset_index()
            df.index = old_index
            columns = df.columns.values
            columns[:2] = ['major','minor']
            df.columns = columns
            return df
        def fix_pos_index(edge_pos,edge_source):
            changed = set(edge_source.index) - set(edge_pos.index)
            edge_source.index = pd.MultiIndex.from_tuples(edge_source.index)
            edge_pos.index = pd.MultiIndex.from_tuples(edge_pos.index)
            for e in changed:
                inv = (e[1],e[0])
                edge_pos.loc[inv,'minor'] = e[1]
                edge_pos.loc[inv,'major'] = e[0]
            return edge_pos
        self.node_source= self.node_mapper.output.join(self.node_tooltip.output)
        #self.edge_tooltip.output.index = pd.MultiIndex.from_tuples(self.edge_tooltip.output.index.values)
        #self.edge_mapper.output.index = pd.MultiIndex.from_tuples(self.edge_mapper.output.index.values)
        self.edge_source= self.edge_mapper.output.join(self.edge_tooltip.output) 
        
        node_pos = self.gc.layout.node['2d'].dropna(axis=1).copy()
        #edge_pos = self.gc.layout.edge['2d'].copy()
        
        self.output_node = pd.concat([self.node_source, node_pos], axis=1).fillna('NaN').copy()
        self.output_node.columns = [str(x) for x in self.output_node.columns]
        self.output_node.index = [str(x) for x in self.output_node.index]
        df = self.edge_source.copy()
        df['src'] = self.gc.edge['src'].values
        df['dst'] = self.gc.edge['dst'].values
        lay = self.gc.layout.edge['2d'].fillna(0).copy()
        #df.loc[:,'index'] = df['index'].map(str).values
        lay = lay.reset_index()
        lay['src'] = lay['index'].map(lambda x:x[0])
        lay['dst'] = lay['index'].map(lambda x:x[1])
        """for c in lay.columns:
            df[c] = np.nan
        rows = lay.iterrows()
        for ix,row in rows:
            cond_src = df['src']==ix[0]
            cond_dst = df['dst']==ix[1]
            df.loc[cond_src&cond_dst,lay.columns] = row.values"""
        self.output_edge = df.merge(lay,on=['src','dst'], left_index=True).fillna('NaN').copy()#edge_pos.merge(self.edge_source,on=['major','minor'],left_index=True)
        def _hack(x):
            try:
                return float(x)
            except:
                return str(x)
        self.output_edge = self.output_edge.applymap(_hack)
    
    def update_mappers(self):
        
        edge_mapper_data = self.gc.edge
        self.edge_mapper.data = edge_mapper_data
        self.node_mapper.data = self.gc.node
        self.node_tooltip.data = self.gc.node
        self.edge_tooltip.data = edge_mapper_data
    
    def update_tooltips(self):
        """Update tooltip data and inject it into the plot tooltips"""
        self.plot.tools[-1].tooltips = self.edge_tooltip.create_tooltip()
        self.plot.tools[-2].tooltips = self.node_tooltip.create_tooltip()

    def update(self, _=None):
        """Set up all the combined elements needed for the plot"""
        self.node_mapper.update()
        self.edge_mapper.update()
        self.update_mappers()
        self.update_source_df()
        self.update_tooltips()
        self.push_data()
        
    def push_data(self):
        """A function to push the content of the source DataFrame
        to a specific plot source
        """
        for col in self.output_node.columns:
            self.node_bokeh_source.data[col] = self.output_node[col].values
        
        for col in self.output_edge.columns:
            self.edge_bokeh_source.data[col] = self.output_edge[col].values
        push_notebook()
    
    def _on_marker_change(self, _=None):
        #clear_output()
        #self.init_plot()
        self.update()
        #return display(show(self.plot))
    
    def init_plot(self):
        """Handle plot init"""
        self.node_bokeh_source = bks.ColumnDataSource(self.output_node)
        self.edge_bokeh_source = bks.ColumnDataSource(self.output_edge)
        n_ttip = self.node_tooltip.create_tooltip()
        e_ttip = self.edge_tooltip.create_tooltip()

        self.plot = figure(width=self._width, height=self._height, webgl=False,
                           tools="pan,wheel_zoom,box_zoom,reset,resize,crosshair,save",)

        #node_marker = 'circle'#self.node_marker_free_sel.marker.value
        #edge_marker = self.edge_marker_free_sel.marker.value
        self.plot.title.text  = self._plot_title
        self.plot.segment('x0', 'y0',
                          'x1', 'y1',
                          color='line_color',
                          line_width='line_width',
                          alpha='line_alpha',
                          source=self.edge_bokeh_source)

        edg_center = self.plot.scatter(source=self.edge_bokeh_source,
                                       x='cx', y='cy',
                                       line_color='line_color',
                                       fill_color='fill_color',
                                       alpha='fill_alpha',
                                       size='size',
                                       line_width='line_width',
                                       marker=self.edge_mapper.marker.marker_type.value
                                       )#,
                                       #marker=edge_marker)
        
        nod = self.plot.scatter(source=self.node_bokeh_source,
                                x='x', y='y',
                                line_color='line_color',
                                fill_color='fill_color',
                                fill_alpha='fill_alpha',
                                size='size',
                                line_width='line_width',#,
                                marker=self.node_mapper.marker.marker_type.value)
    
        if self.plot_ctrl.plot_text.value:
            self.plot.text(source=self.node_bokeh_source,
                           x='x', y='y', y_offset=8,
                           text='label',
                           text_align='center')

        self.plot.add_tools(HoverTool(tooltips=n_ttip, renderers=[nod]))
        self.plot.add_tools(HoverTool(tooltips=e_ttip, renderers=[edg_center]))
        self.plot.title.align = "center"
        self.plot.title.text_font_size = "22px"      
        #no grid 
        self.plot.xgrid.grid_line_color = None
        self.plot.ygrid.grid_line_color = None
        #y axis
        self.plot.yaxis.axis_line_width = 0
        self.plot.yaxis.major_tick_line_color = None
        self.plot.yaxis.major_tick_line_width = 0
        self.plot.yaxis.minor_tick_line_color = None
        #self.plot.yaxis.axis_label_text_font_size = "18px"
        
        #self.plot.yaxis.major_label_text_font_size = "14px"
        #self.plot.yaxis[0].formatter = NumeralTickFormatter(format="0%")
       
        self.plot.yaxis.axis_line_width = 0
        self.plot.yaxis.axis_line_color = None
        self.plot.yaxis.major_label_text_font_size = "0px"
        self.plot.yaxis.axis_label_text_font_size = "0px"
        self.plot.yaxis.axis_label = ''
        #x axis
        self.plot.xaxis.axis_line_width = 0
        self.plot.xaxis.major_tick_line_color = None
        self.plot.xaxis.major_tick_line_width = 0
        self.plot.xaxis.minor_tick_line_color = None
        #self.plot.yaxis.axis_label_text_font_size = "18px"
        
        #self.plot.yaxis.major_label_text_font_size = "14px"
        #self.plot.yaxis[0].formatter = NumeralTickFormatter(format="0%")
       
        self.plot.xaxis.axis_line_width = 0
        self.plot.xaxis.axis_line_color = None
        self.plot.xaxis.major_label_text_font_size = "0px"
        self.plot.xaxis.axis_label_text_font_size = "0px"
        self.plot.xaxis.axis_label = ''
        self._handler = show(self.plot, notebook_handle=True)
    @property
    def snapshot(self, name='bokeh_scatter'):
        html =  notebook_div(self.plot)
        widget = shaoscript('html$N='+name)
        widget.value = html
        return widget

    def load_data(self):
        self.node_mapper.update()
        self.edge_mapper.update()
        self.update_mappers()
        
        self.update_source_df()
        self.init_plot()
        """ self.node_mapper.observe(self.update)
        self.edge_mapper.observe(self.update)
        self.node_tooltip.observe(self.update)
        self.edge_tooltip.observe(self.update)
        self.gc.calculate.observe(self.update)"""
        #self.update_tooltips()
       
    
    def show(self):
        return display(self.widget, show(self.plot))

In [3]:
class GraphCalculator(Dashboard):
    """Handles everything to do with graph managing.
       Attributes:
       ----------
       matrix_panel: pd.Panel of edge metrics/data
       gp: GraphParams object used to feed its graph parameters
       mp: MatrixParams object used to manage its graph's adjacency matrix
       gm: GraphMaker object in charge of graphs related calculations
       widget: GUI for selecting all the params related to graph calculations
    """
    def __init__(self, G,
                 mode='interactive',
                 metrics=False,
                 **kwargs):
        self._directed = G.is_directed()
        self.G = G.to_directed()
        
        self.metrics = metrics
        dash = ['c$N=graphcalculator',
                ['###Graph Calculator$N=gc_title',
                 ['r$N=body_row',[
                                  LayoutCalculator(self.G, name='layout', mode='interactive')
                                 ]
                 ],
                 ['r$N=buttons_row',['btn$d=Calculate',
                                     'tog$N=toggle_gp&d=Creation',
                                     'tog$N=toggle_mp&d=Matrix',
                                     'tog$N=toggle_layout&d=Layout',
                                     'HTML$N=display'
                                    ]
                 ]
                ]
               ]
        
        Dashboard.__init__(self, dash, **kwargs)
        self.display.value = self.display_str('Select and click Calculate')
        
    
        

        self.calculate.observe(self.on_calculate_click)
        self.toggle_layout.observe(self.on_toglay_change)
        self.update()
        self.toggle_layout.value = True
        

    def on_togmp_change(self, change=None):
        self.mp.visible = self.toggle_mp.value

    def on_toggp_change(self, change=None):
        self.gp.visible = self.toggle_gp.value

    def on_toglay_change(self, change=None):
        self.layout.visible = self.toggle_layout.value
        self.layout.button.visible = False

    def display_str(self, s):
        html = '''<div class="graph-gc"
                     style=" font-size:18px;
                            font-weight: bold; 
                            text-align:right;">'''+s+'''</div>'''
        return html

    def on_calculate_click(self, b):
        self.display.value = self.display_str('Calculating...')
        self.update()
        self.display.value = self.display_str('Ready. You can select again.')

    def update(self, _=None):
        def mdg_to_edge_df(g):
            edges = g.edges_iter()
            u0,v0 = next(edges)
            fi_data = g.edge[u0][v0]
            if not  isinstance(g,(nx.MultiDiGraph,nx.MultiGraph)):
                data_cols = list(fi_data.keys())
            else:
                data_cols = list(list(fi_data.values())[0].keys())
            cols = data_cols + ['src','dst']
            
            df = pd.DataFrame(index=np.arange(len(g.edges())),columns=cols)
            df.loc[:,['src','dst']] = np.array(g.edges())
            for c in data_cols:
                if not  isinstance(g,(nx.MultiDiGraph,nx.MultiGraph)):
                    df.loc[0,c] = fi_data[c]
                else:
                    df.loc[0,c] = fi_data[list(fi_data.keys())[0]][c]
            i = 1
            for u,v in edges:
                e_data = g.edge[u][v]
                for c in data_cols:
                    if not  isinstance(g,(nx.MultiDiGraph,nx.MultiGraph)):
                        try:
                            e = e_data[c]
                        except:
                            e = -2
                        
                        df.loc[i,c] = e
                    else:
                        df.loc[i,c] = e_data[list(fi_data.keys())[0]][c]
                i += 1
            return df
        if self.metrics and not  isinstance(self.G,(nx.MultiDiGraph,nx.MultiGraph)):
            if self._directed:
                self.comp = CentralityComputer(self.G)
            else:
                self.comp = CentralityComputer(self.G.to_undirected())
            self.G, self.node = self.add_graph_node_attributes(self.G,
                                                                   self.comp.metrics.T.copy())                                           
        else:
            self.node = pd.DataFrame(self.G.node).T
        self.node = self.node.dropna(axis=1,how='all')
        self.node.loc[:, 'label'] = self.node.index.values
        self.node.loc[:, 'index'] = np.arange(len(self.node.index.values))
        if not  isinstance(self.G,(nx.MultiDiGraph,nx.MultiGraph)):
            self.G = self.add_graph_edge_attributes(self.G)
        #self.edge = pd.Panel(self.G.edge).swapaxes(0, 1)
        self.edge = mdg_to_edge_df(self.G)
        self.layout.update(self.G)

    def add_node_info(self, G, key, pds):
        new_attributes = map(lambda x: (x[0], x[1]), filter(lambda i: G.has_node(i[0]),
                                                            pds.iteritems())
                            )
        nx.set_node_attributes(G, key, dict(new_attributes))

    def add_edge_info(self, G, key, data):
        for i, E in enumerate(G.edges()):
            u, v = E
            G.edge[u][v][key] = data.iloc[i]

    def add_graph_edge_attributes(self, G, func_list=None):
        if func_list is None:
            func_list = [nx.edge_betweenness_centrality, nx.edge_betweenness,
                         nx.edge_current_flow_betweenness_centrality]#,nx.eigenvector_centrality]
        columns = [f.__name__ for f in func_list]
        dft = pd.DataFrame(index=np.arange(len(G.edges())), columns=columns).sort_index()
        for i, name in enumerate(columns):
            try:
                vals = func_list[i](G)
            except:
                continue
            values = list(vals.values())
            dft[name] = values
        for name in columns:
            self.add_edge_info(G, name, dft[name])
        return G

    def add_graph_node_attributes(self, G, metrics):
        for name in metrics.columns:
            self.add_node_info(G, name, metrics[name])
        return G, pd.DataFrame(G.node).T

In [4]:
%%HTML
<style>
.container{width:90% !important;}
</style>

In [5]:
import pickle

with open('G_comp.pck', 'rb') as handle:
    g = pickle.load(handle)


In [11]:
len(g['5370af43e4b0cff95558c12a'].nodes())

106

In [6]:
gm = GraphCalculator(g,metrics=False)

/home/kalidus/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:118: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



In [8]:
gm.node

,E_churn,E_dislikes_sum,E_likes_num,E_likes_std,E_votes_mean,G_interactions_NMF1_d,G_interactions_NMF2_d,G_interactions_w_betweenness_centrality,G_rel_agree_NMF1_d,G_rel_agree_NMF2_d,company,employee,mean_agree,label,index
12_1,0,59,40,3.48982,2.90196,1.88758,0.790732,0.169126,0.726594,0.427767,12,1,0.811278,12_1,0
12_101,0,1,2,0.707107,3.67857,0.352274,0.0658126,0.0880965,0.629342,0,12,101,0.811111,12_101,1
12_103,0,15,7,4.31498,3.12821,0.787725,0.16384,0.133827,0.741451,0,12,103,0.864662,12_103,2
12_104,0,5,1,NaN,3.8,0.0939977,0.211624,0.528093,0.535067,0,12,104,0.846154,12_104,3
12_108,0,1,1,NaN,3.1,0.170411,0,0.540958,0.380573,0,12,108,0.888889,12_108,4
12_12,0,25,18,3.41086,3.06897,0.366508,0.127593,0.154001,0.431654,0.0284437,12,12,0.666667,12_12,5
12_13,0,150,75,4.60505,1.96774,6.97274,3.1496,0.190576,0.450858,0.982002,12,13,0.741691,12_13,6
12_14,0,40,8,4.89898,2.1875,0.274613,0.315985,0.493525,0.590576,0.230711,12,14,0.8775,12_14,7
12_15,0,43,31,2.54846,3,0.764506,0.278148,0.541717,0.550937,0.322565,12,15,0.7375,12_15,8
12_16,0,21,5,2.44949,3.17647,0.0348183,0.238912,0.506505,0.398605,0.0835727,12,16,0.794872,12_16,9


In [9]:
gp = GraphPlotBokeh(gm)
#gp._plot_title = "Company"

In [10]:
gp._plot_title = "Company 12"
gp.load_data()
#gp.init_plot()
gp.widget

In [28]:
snap = gp.snapshot.value

import pickle
with open('geant_data/geant_betweenness_new.pck', 'rb') as handle:
    pickle.dump(snap, handle)

In [29]:
from IPython.core.display import HTML
HTML(snap)

In [40]:
gp.plot.text?